# Challenge Dasa - Dynnamic Programming

In [13]:
import logging
from functools import lru_cache
import matplotlib.pyplot as plt
import csv
import statistics
import time
import tracemalloc

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
exames = [
    {"id": 1, "paciente": "Maria", "medidas": [1.2, 0.8, 0.9]},
    {"id": 2, "paciente": "João", "medidas": [2.1, 1.5, 1.9]},
    {"id": 3, "paciente": "Luiza", "medidas": [0.5, 0.7, 0.6]},
    {"id": 4, "paciente": "Carlos", "medidas": [1.0, 1.2, 1.1]},
    {"id": 5, "paciente": "Heitor", "medidas": [2.5, 2.0, 2.2]},
    {"id": 6, "paciente": "Pedro", "medidas": [0.9, 0.8, 0.85]},
    {"id": 7, "paciente": "Fernanda", "medidas": [1.7, 1.6, 1.8]},
    {"id": 8, "paciente": "Lucas", "medidas": [1.1, 1.0, 0.95]},
    {"id": 9, "paciente": "Roger", "medidas": [2.3, 2.1, 2.4]},
    {"id": 10, "paciente": "Rafael", "medidas": [0.6, 0.7, 0.8]},
    {"id": 11, "paciente": "Vinícius", "medidas": [1.4, 1.2, 1.3]},
    {"id": 12, "paciente": "Felipe", "medidas": [1.9, 2.0, 1.8]},
    {"id": 13, "paciente": "Belisa", "medidas": [0.75, 0.8, 0.7]},
    {"id": 14, "paciente": "Bruno", "medidas": [2.2, 2.0, 2.1]},
    {"id": 15, "paciente": "Jessica", "medidas": [1.3, 1.4, 1.2]},
    {"id": 16, "paciente": "Eduardo", "medidas": [0.9, 1.0, 0.85]},
    {"id": 17, "paciente": "Davi", "medidas": [1.6, 1.7, 1.5]},
    {"id": 18, "paciente": "Thiago", "medidas": [2.5, 2.3, 2.4]},
    {"id": 19, "paciente": "Gabriel", "medidas": [1.0, 1.1, 1.0]},
    {"id": 20, "paciente": "Gustavo", "medidas": [0.8, 0.85, 0.9]},
]

In [6]:
@lru_cache(maxsize=None)
def calcular_volume_cached(tupla_medidas):
  """ Calcula e memoriza o volume da amostra """
  volume = 1
  for m in tupla_medidas:
    volume *= m
  logging.debug(f"Volume calculado: {volume} para {tupla_medidas}")
  return volume

def calcular_volume(medidas):
  """ Wrapper para converter lista em tupla para memoization """
  return calcular_volume_cached(tuple(medidas))

def soma_volumes_recursiva(exames, idx=0):
  """ Soma recursivamente o volume de todas as amostras """
  if idx == len(exames):
    return 0
  volume = calcular_volume(exames[idx]['medidas'])
  return volume + soma_volumes_recursiva(exames, idx + 1)

In [5]:
def filtrar_exames(exames, limite):
  """ Filtra exames com pelo menos uma medida acima do limite """
  filtrados = [exame for exame in exames if max(exame['medidas']) > limite]
  logging.info(f"{len(filtrados)} exames encontrados com medidas acima de {limite}")
  return filtrados

In [7]:
def busca_binaria(exames, id_procurado):
  """ Busca binária por ID do exame """
  exames_ordenados = sorted(exames, key=lambda x: x['id'])
  esquerda, direita = 0, len(exames_ordenados) - 1
  logging.info(f"Iniciando busca binária pelo ID: {id_procurado}")
  while esquerda <= direita:
    meio = (esquerda + direita) // 2
    if exames_ordenados[meio]['id'] == id_procurado:
      logging.info(f"Exame encontrado: {exames_ordenados[meio]}")
      return exames_ordenados[meio]
    elif exames_ordenados[meio]['id'] < id_procurado:
      esquerda = meio + 1
    else:
      direita = meio - 1
  logging.warning(f"Exame com ID {id_procurado} não encontrado.")
  return None

In [8]:
def validar_medidas(exame, referencia=(0.5, 2.5)):
    """ Valida se todas as medidas estão dentro do intervalo de referência """
    return all(referencia[0] <= m <= referencia[1] for m in exame['medidas'])

In [9]:
def analise_estatistica(exames):
  """ Calcula média e desvio padrão das medidas de cada paciente """
  estatisticas = []
  for exame in exames:
    medidas = exame['medidas']
    media = statistics.mean(medidas)
    desvio = statistics.stdev(medidas) if len(medidas) > 1 else 0
    estatisticas.append({'paciente': exame['paciente'], 'media': media, 'desvio': desvio})
  return estatisticas

In [10]:
def exportar_relatorio(exames, nome_arquivo="relatorio_exames.txt"):
  """ Exporta um relatório dos exames para um arquivo de texto """
  with open(nome_arquivo, 'w') as f:
    f.write("Relatório de exames críticos\n")
    f.write("============================\n")
    for exame in exames:
      f.write(f"Paciente: {exame['paciente']} - Medidas: {exame['medidas']} - Volume: {calcular_volume(exame['medidas']):.2f}\n")
    logging.info(f"Relatório exportado com sucesso para {nome_arquivo}")

def exportar_csv(exames, nome_arquivo="relatorio_exames.csv"):
    """ Exporta relatório .csv """
    with open(nome_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Paciente', 'Medidas', 'Volume'])
        for ex in exames:
            writer.writerow([ex['paciente'], ex['medidas'], calcular_volume(ex['medidas'])])
    logging.info(f"Relatório CSV exportado para {nome_arquivo}")

In [11]:
def gerar_grafico_volumes(exames):
    """ Gera gráfico de barras com volumes das amostras """
    nomes = [ex['paciente'] for ex in exames]
    volumes = [calcular_volume(ex['medidas']) for ex in exames]
    plt.figure(figsize=(12, 6))
    plt.bar(nomes, volumes, color='skyblue')
    plt.xlabel('Paciente')
    plt.ylabel('Volume da Amostra')
    plt.title('Volume das Amostras por Paciente')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig("grafico_volumes.png")
    plt.show()
    logging.info("Gráfico de volumes gerado e salvo como 'grafico_volumes.png'.")

In [17]:
print("Sistema de Exames Médicos Iniciados")

try:
  inicio = time.time()
  tracemalloc.start()

  print("\n-----------------------------\n")

  criticos = filtrar_exames(exames, 2.0)
  for critico in criticos:
    print(f"Paciente: {critico['paciente']} - Medidas: {critico['medidas']} - Volume: {calcular_volume(critico['medidas']):.2f}")
  exportar_relatorio(criticos)
  exportar_csv(criticos)

  print("\n-----------------------------\n")

  total_volumes = soma_volumes_recursiva(exames)
  print(f"Volume total das amostras: {total_volumes:.2f}")

  print("\n-----------------------------\n")

  id_para_busca = 13
  exame_encontrado = busca_binaria(exames, id_para_busca)
  if exame_encontrado:
    print(f"Exame encontrado\nPaciente: {exame_encontrado['paciente']} - Medidas: {exame_encontrado['medidas']} Volume: {calcular_volume(exame_encontrado['medidas']):.2f}")
  else:
    raise ValueError("Exame não encontrado.")

  print("\n-----------------------------\n")

  inconsistentes = [ex for ex in exames if not validar_medidas(ex)]
  print(f"Exames inconsistentes: {[ex['paciente'] for ex in inconsistentes]}")

  print("\n-----------------------------\n")

  estatisticas = analise_estatistica(exames)
  for estatistica in estatisticas:
    print(f"Paciente: {estatistica['paciente']} - Média: {estatistica['media']:.2f}, Desvio: {estatistica['desvio']:.2f}")

  print("\n-----------------------------\n")

  atual, pico = tracemalloc.get_traced_memory()
  final = time.time()
  print(f"Tempo de execução: {final - inicio:.3f} segundos")
  print(f"Uso atual da memória: {atual / 10**6:.3f} MB")
  print(f"Pico de uso da memória: {pico / 10**6:.3f} MB")
except Exception as e:
        logging.error(f"Erro na execução: {e}")

Sistema de Exames Médicos Iniciados

-----------------------------

Paciente: João - Medidas: [2.1, 1.5, 1.9] - Volume: 5.99
Paciente: Heitor - Medidas: [2.5, 2.0, 2.2] - Volume: 11.00
Paciente: Roger - Medidas: [2.3, 2.1, 2.4] - Volume: 11.59
Paciente: Bruno - Medidas: [2.2, 2.0, 2.1] - Volume: 9.24
Paciente: Thiago - Medidas: [2.5, 2.3, 2.4] - Volume: 13.80

-----------------------------

Volume total das amostras: 79.08

-----------------------------

Exame encontrado
Paciente: Belisa - Medidas: [0.75, 0.8, 0.7] Volume: 0.42

-----------------------------

Exames inconsistentes: []

-----------------------------

Paciente: Maria - Média: 0.97, Desvio: 0.21
Paciente: João - Média: 1.83, Desvio: 0.31
Paciente: Luiza - Média: 0.60, Desvio: 0.10
Paciente: Carlos - Média: 1.10, Desvio: 0.10
Paciente: Heitor - Média: 2.23, Desvio: 0.25
Paciente: Pedro - Média: 0.85, Desvio: 0.05
Paciente: Fernanda - Média: 1.70, Desvio: 0.10
Paciente: Lucas - Média: 1.02, Desvio: 0.08
Paciente: Roger - Mé